<a href="https://colab.research.google.com/github/WILLIANSCASACOLA/Meus-Codigos/blob/main/Pesquisar_Cep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import requests
import json
from pyspark.sql import SparkSession
import time
import pandas as pd

# Inicia a sessão Spark
spark = SparkSession.builder.appName("ViaCEP").getOrCreate()

In [43]:
# Pede o CEP ao usuário
hoje = time.strftime("%d-%m-%Y")
print(f'O dia de hoje é: {hoje}')
cep = input("Digite o CEP: ")

O dia de hoje é: 22-07-2025
Digite o CEP: 06663330


In [44]:
# Monta a URL com o CEP informado
url = f"https://viacep.com.br/ws/{cep}/json/"

response = requests.get(url)

# Verifica se a resposta foi bem-sucedida
if response.status_code == 200:
    data = response.json()

    # Transforma o dicionário em uma lista de um item (Spark espera uma lista de registros)
    data_list = [data]

    # Cria o DataFrame
    df = spark.createDataFrame(data_list)

    # Mostra o resultado
    df.show(truncate=False)
else:
    print(f"Erro ao acessar a API: {response.status_code}")
    data = None # Set data to None if the API call fails

+---------------+---------+-----------+---+---------+----+-------+----------+-----------------------+-------+-----+---+-------+
|bairro         |cep      |complemento|ddd|estado   |gia |ibge   |localidade|logradouro             |regiao |siafi|uf |unidade|
+---------------+---------+-----------+---+---------+----+-------+----------+-----------------------+-------+-----+---+-------+
|Jardim Paulista|06663-330|           |11 |São Paulo|3736|3522505|Itapevi   |Rua José de Paula Gomes|Sudeste|6551 |SP |       |
+---------------+---------+-----------+---+---------+----+-------+----------+-----------------------+-------+-----+---+-------+



In [45]:
# exportar para csv o arquivo df convertido em df pandas
if data: # Check if data exists before creating DataFrame and exporting
    df_pandas = df.toPandas()
    df_pandas['dia'] = hoje
    df_pandas.to_csv(f'O Cep de Numero {cep} foi exportado em {hoje}.csv', index=False, encoding='utf-8-sig')
else:
    print("Não foi possível exportar o arquivo CSV pois os dados não foram obtidos da API.")

In [46]:
print("Arquivo CSV gerado com sucesso!")

Arquivo CSV gerado com sucesso!


In [47]:
#Mostrar os dados em pandas sem a indexação
df_pandas

,bairro,cep,complemento,ddd,estado,gia,ibge,localidade,logradouro,regiao,siafi,uf,unidade,dia
0,Jardim Paulista,06663-330,,11,São Paulo,3736,3522505,Itapevi,Rua José de Paula Gomes,Sudeste,6551,SP,,22-07-2025
